# Sunrise Housing Case study

## Objective

Identify prospective properties to invest in based on the prediction of the actual value of these properties.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('ggplot')

In [2]:
# load the dataset
inp0 = pd.read_csv('train.csv', header=0)
inp0.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### I - Data understanding

**1. Number of rows, columns and column names**

In [3]:
inp0.shape

(1460, 81)

In [4]:
print(list(inp0.columns))

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

**Observation**<br>
1. Number of rows = 1460. Number of columns = 81.<br>
2. Large number of columns implies that many will need to be dropped or they can be used to derive new metrics.

**2. Drop columns based on definition in data dictionary**

In [5]:
#columns contain irrelevant information based on repetition, uniqueness, data dictionary description
col_drop = ['Id','Condition1','Condition2','BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2','BsmtUnfSF','MiscVal',
            'GarageYrBlt']

**3. Presence of NaN or Null**

In [6]:
#presence of NaN values & Null values
print('NaN : ' , list( [inp0[i].isna().sum() for i in inp0.columns]))
print('Null: ', list( [inp0[i].isnull().sum() for i in inp0.columns]))

NaN :  [0, 0, 0, 259, 0, 0, 1369, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 37, 37, 38, 37, 0, 38, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 690, 81, 81, 81, 0, 0, 81, 81, 0, 0, 0, 0, 0, 0, 0, 1453, 1179, 1406, 0, 0, 0, 0, 0, 0]
Null:  [0, 0, 0, 259, 0, 0, 1369, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 37, 37, 38, 37, 0, 38, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 690, 81, 81, 81, 0, 0, 81, 81, 0, 0, 0, 0, 0, 0, 0, 1453, 1179, 1406, 0, 0, 0, 0, 0, 0]


**Observation**<br>
The number of NaN and Null values are same for all the columns in the dataset. <br>
**Action**<br>
Find columns and the percentage of the NaN values which are non-zero to determine further action.

In [7]:
#many NaN & Null values present; find the columns names for more information and store in a dictionary with percentages
inp0_dict0 = {str(i) : round(((inp0[i].isna().sum()/inp0.shape[0])* 100),2) for i in inp0.columns if inp0[i].isna().sum()>0 }

#sort the dictionary with highest percentages first and print; returns a tuple
inp0_dict1 = sorted( inp0_dict0.items() , key = lambda x : x[1], reverse=True)

#convert tuple to dict
inp0_dict0 = { }
inp0_dict0 = { key:value for key,value in inp0_dict1 }

#print columns with NaN values
print(inp0_dict0)

{'PoolQC': 99.52, 'MiscFeature': 96.3, 'Alley': 93.77, 'Fence': 80.75, 'FireplaceQu': 47.26, 'LotFrontage': 17.74, 'GarageType': 5.55, 'GarageYrBlt': 5.55, 'GarageFinish': 5.55, 'GarageQual': 5.55, 'GarageCond': 5.55, 'BsmtExposure': 2.6, 'BsmtFinType2': 2.6, 'BsmtQual': 2.53, 'BsmtCond': 2.53, 'BsmtFinType1': 2.53, 'MasVnrType': 0.55, 'MasVnrArea': 0.55, 'Electrical': 0.07}


**Observation**<br>
Drop the columns with very high percentage of NaN values ie above 30%. These columns have NA values meaning None for categorical variables implying the absence of the said feature which is very high. For remaining columns, find the common number of rows satisfying them and drop them if required else, impute these columns.

In [8]:
#dropping columns with missing values above 30%
col_drop.extend([key for key,value in inp0_dict0.items() if value >= 30])
inp0.drop(col_drop, axis=1, inplace=True)

**4. Determine columns with high number of zeros or categorical features with values highly favouring one value and add them to drop list**<br>
    Since number of rows are less, none will be removed and the missing values will be imputed.

In [9]:
#find all columns with high number of zeroes(30% of zeros ok in each column) and drop them
inp0_0col = [i for i in inp0.columns if (inp0[i] == 0).sum()>(0.3*(inp0.shape[0]))]
col_drop=[]
col_drop.extend(inp0_0col)

#find categorical features favouring highly only 1 value; highest count is more than 85% of the total rows; no variance
col_drop.extend(
    [i for i in inp0.select_dtypes(include='object').columns if inp0[i].value_counts().max() > (0.8*(inp0.shape[0]))])

inp0.drop(col_drop, axis=1, inplace=True)

In [10]:
inp0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 36 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSSubClass    1460 non-null   int64  
 1   MSZoning      1460 non-null   object 
 2   LotFrontage   1201 non-null   float64
 3   LotArea       1460 non-null   int64  
 4   LotShape      1460 non-null   object 
 5   LotConfig     1460 non-null   object 
 6   Neighborhood  1460 non-null   object 
 7   HouseStyle    1460 non-null   object 
 8   OverallQual   1460 non-null   int64  
 9   OverallCond   1460 non-null   int64  
 10  YearBuilt     1460 non-null   int64  
 11  YearRemodAdd  1460 non-null   int64  
 12  RoofStyle     1460 non-null   object 
 13  Exterior1st   1460 non-null   object 
 14  Exterior2nd   1460 non-null   object 
 15  MasVnrType    1452 non-null   object 
 16  ExterQual     1460 non-null   object 
 17  Foundation    1460 non-null   object 
 18  BsmtQual      1423 non-null 

### II - Data cleaning & manipulation

In [11]:
# copy inital dataset
inp1 = inp0.copy()

**1. Impute missing values for categorical & continous features**<br>
**1.1 Categorical features**

In [12]:
#impute categorial features with mode

#find all the categorial features with missing values
cat_impute = [ str(i) for i in inp1.select_dtypes(include='object') if (inp1[str(i)].isna().sum()>0) ]

#impute the missing values with mode
for i in cat_impute:    
    inp1[str(i)] = inp1[str(i)].fillna(str(inp1[str(i)].mode()[0]))    

#confirm that no categorical feature has any missing value
print([ str(i) for i in inp1.select_dtypes(include='object') if (inp1[str(i)].isna().sum()>0) ])

[]


**1.2 Continuous features**

In [13]:
#impute continuous features with mean

#find all the continuous features with missing values
print([str(i) for i in inp1.select_dtypes(include=['int64','float64']) if(inp1[str(i)].isna().sum()>0)])

['LotFrontage']


In [14]:
#impute the missing values with mean within subset on Neighborhood(assumption that LotFrontage will be similar within same
#neighbourhoods)
inp1['LotFrontage'] = inp1.groupby(by='Neighborhood')['LotFrontage'].apply( lambda x : x.fillna(x.mean()) )
print('Count of null values:',inp1['LotFrontage'].isna().sum())

Count of null values: 0


**Observation**<br>
All values are imputed now for both categorial & continuous features.

**2. Reduce the number of categories within a categorical feature**<br>
Categories which have lesser number of occurences(10%) within each categorical feature can be replaced with a new categorical value named 'Others'. Such categorical values play a limited role in model building as their occurence is very less in comparison to the rest and yet they cannot be ignored. Also, this is done to reduce the number of dummy variables that will be created during data preparation prior to modelling which in turn will reduce the complexity of the model. 

In [15]:
#list of categorical values and their counts
x_dict = {}
x_dict = { str(i):inp1[str(i)].value_counts() for i in inp1.select_dtypes(include='object') } 
for key, value in x_dict.items():
    print(key)
    print(value)

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64
LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64
Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
StoneBr     25
SWISU       25
MeadowV     17
Blmngtn     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: Neighborhood, dtype: int64
HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: HouseStyle, dtype: int64
RoofStyle
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2

In [16]:
#replace the occurence of 10% and less with new value 'Others'
cat_limit = 0.10*inp1.shape[0]
for i in inp1.select_dtypes(include='object'):
    cat_limit_list=[]
    for value,count in inp1[str(i)].value_counts().items():
        if count <= cat_limit:
            cat_limit_list.append(str(value))
    inp1[str(i)] = inp1[str(i)].apply(lambda x : 'Others' if x in cat_limit_list else x)

In [17]:
#list of categorical values and their counts
x_dict = {}
x_dict = { str(i):inp1[str(i)].value_counts() for i in inp1.select_dtypes(include='object') } 
for key, value in x_dict.items():
    print(key)
    print(value)

MSZoning
RL        1151
RM         218
Others      91
Name: MSZoning, dtype: int64
LotShape
Reg       925
IR1       484
Others     51
Name: LotShape, dtype: int64
LotConfig
Inside    1052
Corner     263
Others     145
Name: LotConfig, dtype: int64
Neighborhood
Others     1085
NAmes       225
CollgCr     150
Name: Neighborhood, dtype: int64
HouseStyle
1Story    726
2Story    445
1.5Fin    154
Others    135
Name: HouseStyle, dtype: int64
RoofStyle
Gable     1141
Hip        286
Others      33
Name: RoofStyle, dtype: int64
Exterior1st
VinylSd    515
Others     297
HdBoard    222
MetalSd    220
Wd Sdng    206
Name: Exterior1st, dtype: int64
Exterior2nd
VinylSd    504
Others     338
MetalSd    214
HdBoard    207
Wd Sdng    197
Name: Exterior2nd, dtype: int64
MasVnrType
None       872
BrkFace    445
Others     143
Name: MasVnrType, dtype: int64
ExterQual
TA        906
Gd        488
Others     66
Name: ExterQual, dtype: int64
Foundation
PConc     647
CBlock    634
Others    179
Name: Foundatio

**3 Create derived metrics and drop the columns from which new metric is derived**

**3.1 MoSold, YrSold : join the 2 columns to derive a new metric with month and year**

In [18]:
inp1['mySold'] = inp1['YrSold'].astype('str') + '-' + inp1['MoSold'].astype('str')
col_drop=[]
col_drop.extend(['MoSold','YrSold'])

In [19]:
#convert the year columns into datetime format for further manipulation
inp1['YearBuilt']=inp1['YearBuilt'].apply(lambda x:pd.to_datetime(str(x),format='%Y'))
inp1['YearRemodAdd']=inp1['YearRemodAdd'].apply(lambda x:pd.to_datetime(str(x),format='%Y'))
inp1['mySold']=inp1['mySold'].apply(lambda x:pd.to_datetime(str(x),format='%Y-%m'))

**3.2 Calculate age of property**

**4. Fix the data types of all features**

**4.1 LotFrontage : convert from float64 to int64**

In [20]:
inp1['LotFrontage'] = inp1['LotFrontage'].apply(lambda x : int(round(x,2)))

**4.2 SalePrice : convert from int64 to float64 since it is price**

In [21]:
inp1['SalePrice'] = inp1['SalePrice'].apply(lambda x : float(x))

**5. Neighborhood : category Names is misspelt in dataset according to the data dictionary provided**<br>
Assumption made is that all category values must match the ones in the data dictionary.

In [22]:
inp1['Neighborhood'] = inp1['Neighborhood'].apply(lambda x : x.replace('NAmes','Names'))

In [23]:
#drop columns
inp1.drop(col_drop, axis=1, inplace=True)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3967754633.py, line 2)

In [ ]:
#features after data cleaning & manipulation
inp1.info()

In [ ]:
#year = age old property
# year renovate - year sold ; but if year built = year sold then its 0
#sum = sum of areas
#outlier - LotFrontage